In [1]:
import os
import pathlib

import findspark

prj_dir = pathlib.Path().resolve().parent.parent
spark_home = os.path.join(prj_dir / 'spark-3.5.0-bin-hadoop3')

findspark.init(spark_home)

from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession, SQLContext, Row
from pyspark.sql.functions import col

from apache_log_parser import ApacheAccessLog

In [2]:
conf = SparkConf().setMaster('local[2]').setAppName('FinalExercise').set(
    'spark.executer.memory', '2g'
)

sc = SparkContext(conf=conf)
ssc = StreamingContext(sparkContext=sc, batchDuration=20)

ssc.checkpoint('checkpoint')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 12:46:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Users/audioworkstation/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/spark-3.5.0-bin-hadoop3/python/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [3]:
curr_dir = pathlib.Path().resolve()
logs_dir = os.path.join(curr_dir / 'logs')

log_data = ssc.textFileStream(logs_dir)

In [4]:
access_log_dstream = log_data.map(ApacheAccessLog.parse_from_log_line).filter(lambda line: line is not None)


In [5]:
ip_dstream = access_log_dstream.map(lambda pl: (pl.ip ,1))
ip_count = ip_dstream.reduceByKey(lambda x, y: x + y)

ip_bytes_dstream = access_log_dstream.map(lambda pl: (pl.ip, pl.content_size))
ip_bytes = ip_bytes_dstream.reduceByKey(lambda x, y: x + y)


In [6]:
##### TODO: the ip_bytes_request_count_dstream & ip_bytes_sum_dstream DStreams
# ip_count.join(ip_bytes).pprint()
##########################

In [7]:
# Write Dummy function `internalFunction` for transform() function
# def internal_func(rdd):
#     return rdd
    
####### TODO: Define transformed_access_log_dstream by applying the transform() function 
####### to the access_log_dstream 
# transformed_access_log_dstream = access_log_dstream.transform(internal_func)
###############################

In [8]:
####### TODO: Use the window() function to count data over the access_log_stream
# access_log_dstream.window(windowDuration=40, slideDuration=60).pprint()
###############################

In [9]:
######## TODO: define ip_count_dstream of Ip counts per window using reduceByKeyAndWindow()
# ip_count_dstream = ip_dstream.reduceByKeyAndWindow(
#     lambda x, y: x + y ,
#     windowDuration=40,
#     slideDuration=60,
#     invFunc = lambda x, y: x - y
# )
#############################

# ip_count_dstream.pprint(num=30)

In [10]:
# ip_dstream = access_log_dstream.map(lambda entry: entry.ip)
######### TODO: Use the countByValueAndWindow() function on the ip_dstream
######### to define ip_address_request_count

# ip_address_request_count = ip_dstream.countByValueAndWindow(
#     windowDuration=40,
#     slideDuration=60
# )
# ip_address_request_count.pprint()
#############################

In [11]:
######### TODO: Use the countByWindow() function on the ip_dstream
######### To define ip_address_request_count

ip_address_request_count = ip_dstream.countByWindow(
    windowDuration=40,
    slideDuration=60
)
ip_address_request_count.pprint()
#############################

In [12]:
# This section runs a Running count of response codes using updateStateByKey()
# This basically maintains a running sum , rather than sum in windows

def state_full_sum(new_values, global_sum):
    print(f"NEW VALUES: {new_values}")
    print(f"GLOBAL_SUM: {global_sum}")
    
    return sum(new_values) + (global_sum or 0)

response_code_dstream = access_log_dstream.map(lambda entry: (entry.response_code, 1))
response_code_count_dstream = response_code_dstream.updateStateByKey(state_full_sum)
response_code_count_dstream.pprint()

In [13]:
###### Save the IP address request count using the saveAsTextFiles() function #####

ip_address_request_count.saveAsTextFiles(prefix='output', suffix='txt')
################################

In [14]:
ssc.start()

In [ ]:
ssc.awaitTerminationOrTimeout(timeout=100)

NEW VALUES: [1]                                                     (0 + 1) / 1]
GLOBAL_SUM: None
NEW VALUES: [1]                                                                 
GLOBAL_SUM: None
NEW VALUES: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
GLOBAL_SUM: None


-------------------------------------------
Time: 2023-11-20 12:46:40
-------------------------------------------
('304', 1)
('401', 1)
('200', 14)



NEW VALUES: []
GLOBAL_SUM: 1
NEW VALUES: []
GLOBAL_SUM: 1
NEW VALUES: []
GLOBAL_SUM: 14


-------------------------------------------
Time: 2023-11-20 12:47:00
-------------------------------------------
('304', 1)
('401', 1)
('200', 14)

-------------------------------------------
Time: 2023-11-20 12:47:20
-------------------------------------------



NEW VALUES: []
GLOBAL_SUM: 1
NEW VALUES: []
GLOBAL_SUM: 1
NEW VALUES: []
GLOBAL_SUM: 14


-------------------------------------------
Time: 2023-11-20 12:47:20
-------------------------------------------
('304', 1)
('401', 1)
('200', 14)

